#Lectura del archivo "australian_user_reviews.json", transformaciones y grabación en formato csv

In [164]:
import ast
import pandas as pd

In [165]:
# leemos el arhivo australian_user_reviews.json

def leer_comentarios(arch):
  #Función que lee los comentarios    
  comentarios = []
  for linea in arch.readlines():
    comentarios.append(ast.literal_eval(linea))
  return comentarios

# Abrimos el archivo que contiene los comentarios
with open("../data/australian_user_reviews.json", "r", encoding= "utf-8") as arch:
    # Leemos los comentarios del archivo y los ponemos en el dataframe
    df_comentarios = pd.DataFrame(leer_comentarios(arch))

# Cerramos el archivo
arch.close()

#Mostramos la cabecera del Dataframa
df_comentarios.head(3)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."


In [166]:
#Explotamos la columna "reviews" del dataframe en un nuevo dataframe.
df_comenExpan = df_comentarios.explode('reviews')

#Concatenamos el dataframe original con el nuevo dataframe creado
# y elimina la columna "reviews" del dataframe concatenado
df_comentarios = pd.concat([df_comenExpan.drop(['reviews'], axis=1), df_comenExpan['reviews'].apply(pd.Series)], axis=1)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_5212\784116747.py:6: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df_comentarios = pd.concat([df_comenExpan.drop(['reviews'], axis=1), df_comenExpan['reviews'].apply(pd.Series)], axis=1)


In [167]:
#Ponemos el user_id como índice
df_comentarios = df_comentarios.set_index("user_id")

In [168]:
#Eliminamos una columan que está vacía
df_comentarios.drop(df_comentarios.columns.values[8], axis=1, inplace=True)


In [169]:
df_comentarios.fillna("", inplace= True)

In [170]:
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#Creamos una instancia para el analizador de sentimientos de Vader
sentiment_analyzer = SentimentIntensityAnalyzer()

def categoriza_sentimiento(texto):
    #Función que clasifica los sentimientos en base al calculo 
    # del tono emocional de los comentarios y les asigna un valor de 
    # 0 (malo), 1 (neutral) o 2 (positivo)
    retorno = 1 # Inicia una variable con el valor para la categoría de Neutro
    if(len(texto) > 0):
        sentimiento = sentiment_analyzer.polarity_scores(texto)["compound"]
        if(sentimiento >= 0.33):
           retorno = 2 #Categoría de Positivo
        elif(sentimiento <= -0.33):
            retorno = 0 #Categoría de Negativo

    return retorno

df_comentarios["sentiment_analysis"] = df_comentarios["review"].apply(categoriza_sentimiento)

In [171]:
#Eliminamos la columna review
df_comentarios.drop(["review"], axis=1, inplace= True)

In [173]:
#Grabamos el dataframe como archivo con formato csv
df_comentarios.to_csv("../data/australian_user_reviews.csv", index=False)